# Semantic Search using the Inference API

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/07-inference.ipynb)


Learn how to use the [Inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/inference-apis.html) for semantic search.

# 🧰 Requirements

For this example, you will need:

- An Elastic deployment with minimum **4GB machine learning node**
   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook))
   
- An [OpenAI account](https://openai.com/) is required to use the Inference API with 
the OpenAI service. 

# Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Under **Advanced settings**, go to **Machine Learning instances**
   - You'll need at least **4GB** RAM per zone for this tutorial
   - Select **Create deployment**

# Install packages and connect with Elasticsearch Client

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`

In [ ]:
!pip install elasticsearch

Next, we need to import the modules we need. 🔐 NOTE: getpass enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [ ]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
import getpass
import json
import time

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [ ]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

Confirm that the client has connected with this test:

In [ ]:
print(client.info())

Refer to [the documentation](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect to a self-managed deployment.

Read [this page](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect using API keys.


## Create the inference task

Let's create the inference task by using the [Create inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html).

You'll need an OpenAI API key for this that you can find in your OpenAI account under the [API keys section](https://platform.openai.com/api-keys).

In [ ]:
API_KEY = getpass.getpass('Enter OpenAI API key:  ')

client.inference.put_model(
    task_type="text_embedding",
    model_id="openai_embeddings",
    body={
        "service": "openai",
        "service_settings": {
            "api_key": API_KEY
        },
        "task_settings": {
            "model": "text-embedding-ada-002"
        }
    }
)

## Create an ingest pipeline with an inference processor

Create an ingest pipeline with an inference processor by using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method. Reference the OpenAI model created above to infer against the data that is being ingested in the pipeline.

In [ ]:
client.ingest.put_pipeline(
    id="openai_embeddings", 
    description="Ingest pipeline for OpenAI inference.",
    processors=[
    {
      "inference": {
        "model_id": "openai_embeddings",
        "input_output": {
              "input_field": "plot",
              "output_field": "plot_embedding"
        }
      }
    }
  ]
)

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the machine learning model to be used. In this example, the model ID is set to `openai_embeddings`.
- `input_output`: Specifies input and output fields.
- `input_field`: Field name from which the `dense_vector` representation is created.
- `output_field`:  Field name which contains inference results. 

## Create index

The mapping of the destination index - the index that contains the embeddings that the model will create based on your input text - must be created. The destination index must have a field with the [dense_vector](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html) field type to index the output of the OpenAI model.

Let's create an index named `openai-movie-embeddings` with the mappings we need.

In [ ]:
client.indices.delete(index="openai-movie-embeddings", ignore_unavailable=True)
client.indices.create(
  index="openai-movie-embeddings",
  settings={
      "index": {
          "default_pipeline": "openai_embeddings"
      }
  },
  mappings={
    "properties": {
      "plot_embedding": { 
        "type": "dense_vector", 
        "dims": 1536, 
        "similarity": "dot_product" 
      },
      "plot": { 
        "type": "text" 
      }
    }
  }
)

## Insert Documents (option 1)

Let's insert our example dataset of 12 movies.

In [ ]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append({
        "_index": "openai-movie-embeddings",
        "_source": doc,
    })

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `openai-movie-embeddings` index!")
time.sleep(3)

## Semantic search

After the dataset has been enriched with the embeddings, you can query the data using [semantic search](https://www.elastic.co/guide/en/elasticsearch/reference/current/knn-search.html#knn-semantic-search). Pass a `query_vector_builder` to the k-nearest neighbor (kNN) vector search API, and provide the query text and the model you have used to create the embeddings.

In [ ]:
response = client.search(
    index='openai-movie-embeddings', 
    size=3,
    knn={
        "field": "plot_embedding",
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "openai_embeddings",
                "model_text": "Fighting movie"
            }
        },
        "k": 10,
        "num_candidates": 100
        }
)

for hit in response['hits']['hits']:
    doc_id = hit['_id']
    score = hit['_score']
    title = hit['_source']['title']
    plot = hit['_source']['plot']
    print(f"Score: {score}\nTitle: {title}\nPlot: {plot}\n")